In [1]:
import os
import multiprocessing

from json import loads

# from time import sleep
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import kblab 
import math
from pandas.core.frame import DataFrame
from tqdm import tqdm
from urllib3.util import Retry
from urllib3 import PoolManager, make_headers
from kblab import Archive
import regex as re
from itertools import product
kblab.VERIFY_CA=False

In [2]:
years = range(1920, 1930)

In [3]:
df_content: DataFrame = pd.read_feather(f"data/df_content_new_{years[0]}s.feather")

In [ ]:
#df_content: DataFrame = pd.read_feather(f"/data/birdNewsData/data/df_content_new_{years[0]}s.feather")

FileNotFoundError: [Errno 2] No such file or directory: '/data/birdNewsData/data/df_content_new_1920s.feather'

In [4]:
df_content
df_numpy = np.array(df_content)

In [5]:
regex = r"\p{L}+"
def tokenize(text):
  return re.finditer(regex, text.lower())

In [6]:
def text_to_idx(words):
  wordCount = 0
  wordPos = {}
  for token in words:
    wordCount+=1
    word = token.group()
    pos = token.span()[0]
    if word in wordPos.keys():
      wordPos[word].append(pos)
    else:
      wordPos[word] = [pos]
  return wordPos, wordCount

In [7]:
tokens = tokenize("Men hallå där!")
idx, wordCount = text_to_idx(tokens)
print(wordCount)
idx.keys()

3


dict_keys(['men', 'hallå', 'där'])

In [8]:
def getDeclension(word):
  if word[-2:] == "ås":
    root = word[:-2]
    return [word, f"{word}en", f"{root}äss", f"{root}ässen"]
  if word[-2:] == "tt" or word[-2:] == "st":
    root = word[:-1]
    return [word, f"{word}en", f"{word}ar", f"{word}arna"]
  if word[-2:] == "rt":
    root = word[:-1]
    return [word, f"{word}en", f"{word}ar", f"{word}arna"]
  if word[-1:] == "t":
    root = word[:-1]
    return [word, f"{word}en", f"{word}er", f"{word}erna"]
  if word[-3:] == "gam":
    return [word, f"{word}en", f"{word}ar", f"{word}arna"]
  if word[-1:] == "m":
    return [word, f"{word}men", f"{word}mar", f"{word}marna"]
  if word[-2:] == "ss":
    return [word, f"{word}en", f"{word}er", f"{word}erna"]
  if word[-1:] == "a":
    root = word[:-1]
    return [word, f"{root}an", f"{root}or", f"{root}orna"]
  if word[-2:] == "yr":
    root = word[:-2]
    return [word, f"{word}en", f"{root}rar", f"{root}rarna"]
  if word[-3:] == "ger":
    root = word[:-2]
    return [word, f"{root}ern", f"{root}rar", f"{root}rarna"]
  if word[-1:] == "r" and (not word[-2:] == "är"):
    root = word[:-1]
    return [word, f"{word}et", f"{word}", f"{word}en"]
  if word[-1:] == "d":
    root = word[:-3]
    return [word, f"{word}et", f"{root}änder", f"{root}änderna"]
  if word[-3:] == "are" and (not word[-5:] == "stare"):
    root = word[:-1]
    return [word, f"{root}en", f"{root}e", f"{root}na"]
  if word[-2:] == "re":
    root = word[:-1]
    return [word, f"{root}en", f"{root}ar", f"{root}arna"]
  if word[-1:] == "e":
    root = word[:-1]
    return [word, f"{root}en", f"{root}ar", f"{root}arna"]
  if word[-4:] == "rell":
    root = word[:-2] 
    return [word, f"{word}en", f"{word}er", f"{word}erna"]
  if word[-2:] == "ll":
    root = word[:-2] 
    return [word, f"{word}en", f"{word}ar", f"{word}arna"]
  if word[-2:] == "yl" or word[-3:] == "nal":
    root = word[:-2] 
    return [word, f"{word}en", f"{word}er", f"{word}erna"]
  if word[-3:] == "gal":
    root = word[:-2] 
    return [word, f"{word}en", f"{word}ar", f"{word}arna"]
  if word[-1:] == "l":
    root = word[:-2] 
    return [word, f"{root}eln", f"{root}lar", f"{root}larna"]
  if word[-2:] == "ag":
    root = word 
    return [word, f"{root}et", f"{root}", f"{root}en"]
  if word[-1:] == "g":
    root = word 
    return [word, f"{root}en", f"{root}ar", f"{root}arna"]
  if word[-2:] == "um":
    root = word[:-2] 
    return [word, f"{root}en", f"{root}nar", f"{root}narna"]
  if word[-2:] == "en":
    root = word[:-2] 
    return [word, f"{root}nen", f"{root}er", f"{root}erna"]
  if word[-2:] == "an":
    root = word[:-2] 
    return [word, f"{word}en", f"{word}er", f"{word}erna"]
  if word[-2:] == "ur" or word[-2:] == "är":
    root = word[:-2] 
    return [word, f"{word}en", f"{word}er", f"{word}erna"]
  if word[-2:] == "nd":
    root = word[:-3]
    return [word, f"{word}en", f"{root}änder", f"{word}änderna"]
  if word[-1:] == "ö":
    return [word, f"{word}n", f"{word}ar", f"{word}arna"]
  if word[-2:] == "es":
    return [word, f"{word}en", f"{word}ar", f"{word}arna"]
  if word[-2:] == "rk" or word[-2:] == "nk":
    return [word, f"{word}en", f"{word}ar", f"{word}arna"]
  if word[-2:] == "ök" or word[-2:] == "åk" or word[-2:] == "lk" or word[-2:] == "ck":
    return [word, f"{word}en", f"{word}ar", f"{word}arna"]
  if word[-1:] == "k":
    return [word, f"{word}en", f"{word}er", f"{word}erna"]
  if word[-1:] == "o":
    return [word, f"{word}n", f"{word}er", f"{word}rna"]
  if word[-1:] == "s" and (not word[-2:] == "is" or not word[-2] == "es"):
    return [word, f"{word}et", f"{word}", f"{word}en"]
  else:
    root = word
    return [word, f"{root}en", f"{root}ar", f"{root}arna"]



In [11]:
getDeclension("gås")

['gås', 'gåsen', 'gäss', 'gässen']

In [9]:
birds = pd.read_csv("birds2.csv", header=0)
birdsDia = pd.read_csv("birdsDialectic.csv", header=0)
natureTypes = pd.read_csv("naturtyper.csv", header=0)
wordChecklist = natureTypes.copy()
natureTypes["naturtyp"].values

array(['sandbank', 'estuarium', 'sandbotten', 'lerbotten',
       'blottade ler- och sandbotten', 'lagun', 'vik', 'sund', 'rev',
       'bubbelstruktur', 'driftvall', 'stenvall ', 'grusvall',
       'havsklippa', 'strand', 'glasörtstrand', 'salta strandäng', 'ö',
       'åsö', 'skär', 'strandäng', 'smala östersjövik', 'dyn', 'fördyn',
       'vita dyn', 'grå dyn', 'risdyner', 'sandvidedyn', 'trädklädda dyn',
       'dynvåtmark', 'hed', 'rissandhed', 'grässandhedar', 'sjö',
       'slättssjö', 'näringsfattiga slättssjöar', 'ävjestrandsjö',
       'kransalgssjö', 'näringsrika sjö', 'näringsrik sjö',
       'naturligt näringsrika sjö', 'myrsjö', 'vattendrag',
       'större vattendrag', 'alpina vattendrag', 'mindre vattendrag',
       'fukthed', 'torra hed', 'alpina rished', 'alpina videbuskmark',
       'enbuskmark', 'berghäll', 'basiska berghäll', 'sandstäpp', 'mark',
       'gräsmark', 'silikatgräsmark', 'alpina silikatgräsmark',
       'alpina kalkgräsmark', 'kalkgräsmark', 'stagg-grä

In [10]:
birdsDia

,namn,riktigaNamn
0,ackspett,hackspett
1,adfågel,åd(a)
2,aftonfalk,aftonfalk
3,aftonfågel,taltrast
4,aikamies,tofsmes
...,...,...
4145,örra,alfågel
4146,örr,se orre
4147,örre,se orre
4148,östkucku,se under gök


In [11]:
def addWords(word, onlyDeclension = True): 
    global wordChecklist
    if onlyDeclension:
        wordDec = getDeclension(word)[1:]
        wordDecFrame = pd.DataFrame([wordDec[0],wordDec[1],wordDec[2]], columns=["naturtyp"])
    else:
        wordDec = getDeclension(word)
        wordDecFrame = pd.DataFrame([wordDec[0],wordDec[1],wordDec[2],wordDec[3]], columns=["naturtyp"])
    wordChecklist = pd.concat([wordChecklist,wordDecFrame])


In [12]:
for x in natureTypes["naturtyp"].values:
    addWords(x)

for x in birds["namn"].values:
    addWords(x, False)

In [13]:
pd.set_option('display.max_rows', 20)
wordChecklist

,naturtyp
0,sandbank
1,estuarium
2,sandbotten
3,lerbotten
4,blottade ler- och sandbotten
...,...
3,doppingfåglarna
0,seglarfågel
1,seglarfågeln
2,seglarfåglar


In [13]:
from transformers import AutoModel,pipeline

ner = pipeline('ner', model='KB/bert-base-swedish-cased-ner', tokenizer='KB/bert-base-swedish-cased-ner')
pos = pipeline("token-classification", model="KBLab/bert-base-swedish-cased-pos", tokenizer="KBLab/bert-base-swedish-cased-pos")
sentiment = pipeline("text-classification", model="KBLab/robust-swedish-sentiment-multiclass")




/home/tmpuser/code/Exjobb-main/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-01 14:40:41.651565: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.


KeyboardInterrupt: 

In [15]:
def getVerbs(text):
    posText = pos(text)
    l = []
    for token in posText:
        try:
            if token['word'].startswith('##'):
                l[-1]['word'] += token['word'][2:]
            else:
                l += [ token ]
        except:
            print(f"failed with {token}")
    return [x["word"] for x in list(filter(lambda d: d['entity'] == "VB", l))]

In [16]:
def getNamed(text):
    interestingTags = ["PRS", "LOC", "ORG", "OBJ"]
    posText = ner(text)
    l = []
    for token in posText:
        try:
            if token['word'].startswith('##'):
                l[-1]['word'] += token['word'][2:]
            else:
                l += [ token ]
        except:
            print(f"failed with {token}")
    return [x["word"] for x in list(filter(lambda d: d['entity'] in interestingTags, l))]    

In [14]:
wordChecklist

,naturtyp
0,sandbank
1,estuarium
2,sandbotten
3,lerbotten
4,blottade ler- och sandbotten
...,...
3,doppingfåglarna
0,seglarfågel
1,seglarfågeln
2,seglarfåglar


In [15]:
def concatDictDF(dict, pastDf):
    return pastDf.combine_first(pd.DataFrame.from_dict(dict))
#    return pd.concat([pastDf,pd.DataFrame.from_dict(dict)],axis=1)

In [16]:
master_index = {}
filtered_count_index = {}
yearCounts = {}

for y in years:
    yearCounts.update({str(y):0})
totalCount = 0


""" def addToIndex(row):
    tmp = 0
    global master_index
    global filtered_count_index
    tokens = tokenize(row["content"])
    idx, wordCount = text_to_idx(tokens)
    year = row["created"][:4]
    yearCounts[year] += wordCount
    for word in idx.keys():
        if word in master_index:
            master_index[word][row["dark_id"], year] = idx[word]
        else:
            master_index[word] = {(row["dark_id"], year):idx[word]}
        if word in wordChecklist["naturtyp"].values:
              filtered_entry = {word:{row["dark_id"]:len(idx[word])}}
              filtered_count_index = concatDictDF(filtered_entry,filtered_count_index)
                     """
        
def addToIndex(row, saveIndex):
    tmp = 0
    global master_index
    global filtered_count_index
    tokens = tokenize(row["content"])
    idx, wordCount = text_to_idx(tokens)
    year = row["created"][:4]
    yearCounts[year] += wordCount
    for word in idx.keys():
        #if word in master_index:
        #    master_index[word][row["dark_id"], year] = idx[word]
        #else:
        #    master_index[word] = {(row["dark_id"], year):idx[word]}
        if saveIndex == 1:
          if word in wordChecklist["naturtyp"].values:
#                filtered_entry = {word:{row["dark_id"]:len(idx[word])}}
                
                if word in filtered_count_index:               
                  filtered_count_index[word][row["dark_id"]] = len(idx[word])
                else:
                  #master_index[word] = {(row["dark_id"], year):idx[word]}
                  filtered_count_index[word] = {row["dark_id"]:len(idx[word])}


In [17]:
len(filtered_count_index)

0

In [18]:
df_content[:10].apply(lambda row: addToIndex(row,1), axis=1)

TypeError: addToIndex() missing 1 required positional argument: 'saveIndex'

In [26]:
filtered_count_index.sort_values(by="uggla",ascending=False)

,albatross,alfågel,alfåglarna,and,barrskog,barrskogar,betesmark,blåmes,bofink,bofinkar,...,älven,änder,äng,ängar,ängarna,ängen,ö,öar,öarna,ön
dark-102395,2.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,1.0,NaN,35.0,NaN,1.0,1.0
dark-102406,NaN,NaN,NaN,5.0,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,NaN,39.0,NaN,NaN,1.0
dark-102409,NaN,NaN,NaN,8.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,36.0,2.0,NaN,5.0
dark-102405,NaN,NaN,NaN,5.0,2.0,NaN,NaN,1.0,1.0,1.0,...,4.0,3.0,NaN,1.0,2.0,NaN,31.0,2.0,2.0,21.0
dark-102404,NaN,NaN,NaN,8.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,34.0,2.0,NaN,2.0
dark-102398,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,1.0,42.0,NaN,1.0,3.0
dark-102407,NaN,NaN,NaN,8.0,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,35.0,1.0,NaN,NaN
dark-102408,NaN,NaN,NaN,13.0,2.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,1.0,2.0,1.0,47.0,1.0,2.0,4.0
dark-102412,NaN,1.0,2.0,2.0,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,44.0,NaN,3.0,2.0
dark-102396,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,1.0,NaN,1.0,NaN,33.0,NaN,1.0,1.0


In [27]:
len(master_index)

74555

In [28]:
def indices(lst, element):
    result = []
    offset = -1
    while True:
        try:
            offset = lst.index(element, offset+1)
        except ValueError:
            return result
        result.append(offset)

In [33]:
docs = []
def getDocs(word):
    global docs
    master_index_bird = master_index[word]
    for issue, year in master_index_bird:
        res = df_content.loc[df_content["dark_id"] == issue]
        res = res["content"].values
        textBlock = res[0].lower().replace(",", "")
        n = 10
        idxs = indices(textBlock, word)
        for id in idxs:
            lhs, bird, rhs = textBlock[id-50:id+50].partition(word)
            sentimentWindow = " ".join(lhs.split()[-n:] + [bird] + rhs.split()[:n])
            docs.append(sentimentWindow)

In [34]:
for i, b in filtered_count_index.items():
    getDocs(i)

In [36]:
len(docs)

89345

In [37]:
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)


topic_model = BERTopic(embedding_model='KBLab/sentence-bert-swedish-cased', verbose=True, ctfidf_model=ctfidf_model)
topics, probs = topic_model.fit_transform(docs)

embedding_model = 'KBLab/sentence-bert-swedish-cased'
topic_model.save("ownData/", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

2024-07-02 17:04:09,365 - BERTopic - Embedding - Transforming documents to embeddings.


Batches: 100%|██████████| 2793/2793 [01:18<00:00, 35.42it/s]
2024-07-02 17:05:31,775 - BERTopic - Embedding - Completed ✓
2024-07-02 17:05:31,778 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-07-02 17:06:55,860 - BERTopic - Dimensionality - Completed ✓
2024-07-02 17:06:55,864 - BERTopic - Cluster - Start clustering the reduced embeddings
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PAR

In [39]:
topic_term_matrix = topic_model.c_tf_idf_
words = topic_model.vectorizer_model.get_feature_names()

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names'

In [40]:
similar_topics, similarity = topic_model.find_topics("fågel", top_n=5)
topic_model.get_topic(similar_topics[0])

[('flugsnappare', 0.24957560984977473),
 ('hussvala', 0.24634214727174225),
 ('ladusvala', 0.24634214727174225),
 ('grönsiska', 0.24634214727174225),
 ('fåglar', 0.24511798954045613),
 ('blåmes', 0.2429227390481649),
 ('fröätande', 0.23929740580467726),
 ('trädkrypare', 0.2354426768370055),
 ('svartmes', 0.2354426768370055),
 ('grönfink', 0.23133070496419383)]

In [101]:
tmpdf = pd.DataFrame()
tmp12 = pd.DataFrame.from_dict({"ö":{"dark-102238":50}})
tmp13 = pd.DataFrame.from_dict({"ö":{"dark-1021238":20}})
tmp15 = pd.DataFrame.from_dict({"fåggel":{"dark-102238":50}})
tmp14 = pd.DataFrame.from_dict({"fåggel":{"dark-1021238":20}})
tmpdf = concatDictDF(tmp12,tmpdf)
tmpdf = concatDictDF(tmp13,tmpdf)
tmpdf = concatDictDF(tmp15,tmpdf)
tmpdf = concatDictDF(tmp14,tmpdf)
print(tmpdf)


              fåggel   ö
dark-1021238    20.0  20
dark-102238     50.0  50


In [51]:
filtered_count_index["uggla"].sum()

22.0

In [135]:
def counter(word):
  master_index_bird = master_index[word]
  count = {}

  for y in years:
    count.update({str(y): {"freq":0, "count":0, "prob":0}})
  for a,b in master_index_bird:
    if b in count:
      count[b]["count"] += 1
    else:
      count[b]["count"] = 1
  for y in years:
    try:
      count[str(y)]["freq"] = count[str(y)]["count"]/yearCounts[str(y)]*100000
      count[str(y)]["prob"] = count[str(y)]["count"]/yearCounts[str(y)]
    except:
      count[str(y)]["freq"] = 0.0
  return count


In [136]:
def getWordCooccurenceAndSentiment(word):
  yearsString = [str(y) for y in years]
  master_index_bird = master_index[word]
  sentimentByYear = {}
  global df_numpy
  cooccurence = {}
  for y in years:
    cooccurence.update({str(y):{}})
    sentimentByYear.update({str(y):[]})
  for issue, year in master_index_bird:
    rows, cols = np.where(df_numpy == issue)
    textBlock = df_numpy[rows][0][0].lower().replace(",", "")
    n = 5
    lhs, bird, rhs = textBlock.partition(word)
    window = lhs.split()[-n:] + rhs.split()[:n]
    sentimentWindow = " ".join(lhs.split()[-n:] + [bird] + rhs.split()[:n])
    textSentiment = sentiment(sentimentWindow)[0]["label"]
    sentimentByYear[year].append(textSentiment)
    birdWords = list(filter(None, window))
    #verbs = getVerbs(" ".join(birdWords))
    #named = getNamed(tmp)
    #natureWords = [k for k in birdWords if k in wordChecklistList]
    #interestingWords = verbs + natureWords
    #print(f"all: {birdWords}")
    #print(f"verbs {verbs}")
    #print(f"nature: {natureWords}")
    for bWord in birdWords:
      if bWord == word or bWord in yearsString:
        continue
      if bWord in cooccurence[year]:
        cooccurence[year][bWord] += 1
      else:
        cooccurence[year][bWord] = 1
      #print(f"checking {bWord}")    
  return cooccurence, sentimentByYear

In [137]:
def sortCooccurence(a):
  return sorted(a.items(), key=lambda x:x[1], reverse=True)

In [138]:
def mergeCounts(c1, c2):
  tmp = {}
  for y in years:
    tmp[str(y)] = {"freq": c1[str(y)]["freq"] + c2[str(y)]["freq"], "count": c1[str(y)]["count"] + c2[str(y)]["count"], "prob": c1[str(y)]["prob"] + c2[str(y)]["prob"]}
  return tmp


In [139]:
def mergeCoocs(c1, c2):
  tmp = {}
  for y in years:
    tmp[str(y)] = {k: (c1[str(y)].get(k, 0) + c2[str(y)].get(k, 0)) for k in set(c1[str(y)]) | set(c2[str(y)])}
  return tmp

In [140]:
def mergeSentiment(s1,s2):
    tmp = {}
    for y in years:
        tmp1 = s1[str(y)]
        tmp2 = s2[str(y)]
        tmp[str(y)] = tmp1+tmp2
    return tmp

In [141]:

frequency = {}
cooccurenceMatrix = {}

for i, bird in birds.iterrows():
  specificbird = bird["namn"]
  birds_dec = getDeclension(specificbird)
  for bd in birds_dec:
    try:
      bc = counter(bd)
      if bird[0] in frequency:
        freq = frequency[specificbird]
      else:
        freq = {}
        for y in years:
          freq.update({str(y):{"freq":0, "count":0, "prob":0}})
      frequency[specificbird] = mergeCounts(bc, freq)
    except:
      "lmao"

/tmp/ipykernel_906853/3438062497.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if bird[0] in frequency:


In [142]:
frequency.keys()
dataFreq = {}
dataCount = {}

for b in frequency.keys():
    dataFreq.update({b: [frequency[b][x]["freq"] for x in frequency[b]]})
    dataCount.update({b: [frequency[b][x]["count"] for x in frequency[b]]})




In [143]:
dfFreq = pd.DataFrame.from_dict(dataFreq, orient="index", columns=years)
dfCount = pd.DataFrame.from_dict(dataCount, orient="index", columns=years)
dfFreq.to_feather(f"/data/birdNewsData/birdFreq/dfFreq_{years[0]}s.feather")
dfCount.to_feather(f"/data/birdNewsData/birdFreq/dfCount_{years[0]}s.feather")

In [144]:
pd.DataFrame.from_dict(yearCounts, orient="index", columns=["word count"]).to_feather(f"/data/birdNewsData/birdFreq/dfYearCount_{years[0]}s.feather")

In [145]:
pd.read_feather(f"/data/birdNewsData/birdFreq/dfFreq_{years[0]}s.feather")

,1930,1931,1932,1933,1934,1935,1936,1937,1938,1939
fågel,0.680119,0.0,2.331702,0.0,0.0,0.0,1.446801,0.0,10.181226,0.0
andfågel,0.000000,0.0,0.291463,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
gås,0.000000,0.0,0.582926,0.0,0.0,0.0,5.787204,0.0,0.000000,0.0
svan,0.680119,0.0,0.874388,0.0,0.0,0.0,1.446801,0.0,3.393742,0.0
and,2.040358,0.0,2.040239,0.0,0.0,0.0,2.893602,0.0,3.393742,0.0
...,...,...,...,...,...,...,...,...,...,...
gråsparv,0.000000,0.0,0.291463,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
ärla,0.680119,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
domherre,0.000000,0.0,0.000000,0.0,0.0,0.0,1.446801,0.0,0.000000,0.0
kanariefågel,0.000000,0.0,0.000000,0.0,0.0,0.0,2.893602,0.0,3.393742,0.0


In [146]:

for i, bird in birds.iterrows():
    specificbird = bird["namn"]
    print(specificbird)
    birds_dec = getDeclension(specificbird)
    print(birds_dec)

    cooccurenceMatrix = {}
    sentimentMatrix = {}
    for bd in birds_dec:
        print(f"{bd} start")
        coWord, senWord = getWordCooccurenceAndSentiment(bd)
        if specificbird in cooccurenceMatrix:
            co = cooccurenceMatrix[specificbird]
            cooccurenceMatrix[specificbird] = mergeCoocs(co, coWord)
        else:
            cooccurenceMatrix[specificbird] = coWord

        if specificbird in sentimentMatrix:
            se = sentimentMatrix[specificbird]
            sentimentMatrix[specificbird] = mergeSentiment(se, senWord)
        else:
            sentimentMatrix[specificbird] = senWord
        print(f"{bd} done")
    
    pd.DataFrame.from_dict(cooccurenceMatrix[specificbird]).to_feather(f"/data/birdNewsData/birdFreq/df_co_{years[0]}s_{specificbird}.feather")
    specificSentiment = sentimentMatrix[specificbird]
    tmp = {}
    for y in years:
        yearlySentiment = specificSentiment[str(y)]
        countNegative = yearlySentiment.count("NEGATIVE")
        countPositive = yearlySentiment.count("POSITIVE")
        CountNeutral = yearlySentiment.count("NEUTRAL")
        tmp[y] = {"Positive": countPositive,"Neutral":CountNeutral, "Negative" : countNegative}
    pd.DataFrame.from_dict(tmp).to_feather(f"/data/birdNewsData/birdFreq/df_sen_{years[0]}s_{specificbird}.feather")
    #break

        



fågel
['fågel', 'fågeln', 'fåglar', 'fåglarna']
fågel start


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:

def getCoWordsCount(bird):
    tmp = 0
    specificMatrix = cooccurenceMatrix[bird]
    SavedCounts = {}
    for y in years:
        for x in specificMatrix[str(y)].keys():
            if not (x in SavedCounts):
                try:
                    tmp1 = [counter(x)[str(y)]["count"] for y in years]
                    SavedCounts[x] = tmp1
                except:
                    "not in index..."
    return SavedCounts

: 

In [ ]:
tmpFa = getCoWordsCount("fågel")

: 

In [ ]:
tmpFa

: 

In [ ]:
df_coCount = pd.DataFrame.from_dict(tmpFa, orient="index", columns=years)
df_coCount.to_feather(f"/data/birdNewsData/birdFreq/df_coWord_{years[0]}s_fågel.feather")


: 

In [ ]:
pd.read_feather(f"/data/birdNewsData/birdFreq/df_coWord_{years[0]}s_fågel.feather")


: 

In [ ]:
def getTScore(bird, year):
  ts = {}
  skipwords = ["och", "i", "en", "som", "att", "kan", "den", "ett"]
  for word, count_co in cooccurenceMatrix[bird][year].items():
    if word in skipwords:
      continue
    try:
      count_word = counter(word)[year]["count"]
      prob_word = count_word/yearCounts[year] *100000
      prob_bird = frequency[bird][year]["freq"]
      prob_both = count_co/yearCounts[year] * 100000

      ts[word] = ((
        prob_both -
        prob_bird *
        prob_word /
        yearCounts[year] /
        math.sqrt(prob_both)
      ))
    except:
      "lmao"
  return sorted(ts.items(), key=lambda x:x[1], reverse=True)[:10]
  

: 

In [ ]:
def getPMIs(bird, year):
  tmp = []
  count_bird = frequency[bird][year]["count"]
  #print("count b\t", count_bird)
  prob_bird = frequency[bird][year]["prob"]
  #print("prob b\t", prob_bird)
  i = 0
  for word, count_co in cooccurenceMatrix["Domherre"][year].items():
    try:
      print(word)
      i+=1
      if i > 5:
        break; 
      count_word = counter(word)[year]["count"]
      #print("count w\t", count_word)
      prob_word = count_word/yearCounts[year]
      #print("prob w\t", prob_word)
      #print("count co w\t", count_co)
      prob = count_co/yearCounts[year]
      #print("total prob\t", prob)
      pmi = max(math.log(prob/(prob_bird*prob_word)),0)
      #print("pmi\t", pmi)
      tmp.append((word, pmi))
      #print()
    except:
      "oof"
  return sorted(tmp, key=lambda x:x[1], reverse=True)

: 

In [ ]:
tmp = 0

for bird, freq in frequency.items():
  tmp +=1
  onlyFreqs = {}
  avg = 0
  for f in freq:
    onlyFreqs.update({str(f): freq[f]["freq"]})
    avg += freq[f]["freq"]/10
  if avg > 1:
    plt.plot(onlyFreqs.keys(), onlyFreqs.values(), label=bird)



plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.tight_layout()
plt.show()


: 